# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("../weather_data_df.csv", encoding="utf-8")
del weather_df["Unnamed: 0"]
weather_df.head()

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,castro,-24.79,-50.01,62.53,80,98,6.71,BR,1596036790
1,klaksvik,62.23,-6.59,50.00,87,100,19.46,FO,1596036793
2,akyab,20.15,92.90,82.83,85,100,8.08,MM,1596036795
3,rikitea,-23.12,-134.97,72.57,80,11,11.59,PF,1596036771
4,upernavik,72.79,-56.15,43.32,91,100,4.32,GL,1596036799


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()

In [6]:
fig = gmaps.figure(center=(30.0, -10.0), zoom_level=2)
heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=2)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_df = weather_df.loc[(weather_df["Temperature"] > 70) & (weather_df["Temperature"] <80) & (weather_df["Wind Speed"] < 3), :]
ideal_df = ideal_df.dropna(how="any")

ideal_df.reset_index(inplace=True)
del ideal_df["index"]

ideal_df

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,ambilobe,-13.20,49.05,74.91,76,24,1.66,MG,1596036990
1,kuching,1.55,110.33,77.00,94,40,2.24,MY,1596037013
2,xinye,32.52,112.35,77.97,87,100,2.33,CN,1596037069
3,coahuayana,18.73,-103.68,75.20,88,40,2.71,MX,1596037261
4,tongren,27.72,109.19,75.72,87,37,1.12,CN,1596037438
5,ambon,-3.70,128.18,78.01,15,90,1.01,ID,1596037463
6,kijang,35.24,129.21,75.20,94,75,1.12,KR,1596037547
7,puerto asis,0.51,-76.50,71.60,96,100,0.25,CO,1596037693
8,gambela,8.25,34.58,74.32,89,62,0.40,ET,1596037798
9,saint-jean-de-maurienne,45.17,6.58,79.00,49,61,1.99,FR,1596037899


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Create hotel list
hotel_list = []

#Create loop for hotel list
for city in range(len(ideal_df)):
    lat = ideal_df.loc[city]["Lat"]
    lng = ideal_df.loc[city]["Lng"]
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    jsn = response.json()
    try:
        hotel_list.append(jsn["results"][0]["name"])
    except:
        hotel_list.append("")

ideal_df["Hotel Name"] = hotel_list

ideal_df = ideal_df.dropna(how="any")
ideal_df.head()

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,ambilobe,-13.20,49.05,74.91,76,24,1.66,MG,1596036990,Ambilobe
1,kuching,1.55,110.33,77.00,94,40,2.24,MY,1596037013,Kuching
2,xinye,32.52,112.35,77.97,87,100,2.33,CN,1596037069,Nanyang
3,coahuayana,18.73,-103.68,75.20,88,40,2.71,MX,1596037261,Coahuayana de Hidalgo
4,tongren,27.72,109.19,75.72,87,37,1.12,CN,1596037438,Tongren


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))